In [4]:
import pandas as pd
# from factor_analyzer import FactorAnalyzer
# from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity, calculate_kmo
import numpy as np

data_path = "G:\ABCD\script/trail/trail_tsne_RF"
# load data and drop the first column and the subject id
data_raw = pd.read_csv(data_path + "/merged.csv")
label_columns = data_raw.columns[data_raw.columns.str.startswith("cbcl")].tolist()

data_raw = data_raw[["src_subject_id"] + label_columns]

#out put directory
rotation = "oblimin"
output_dir = "G:/ABCD/script/trail/trail_tsne_RF/factor analysis/output/" + rotation


In [71]:
#drop the subject id for data_raw
data = data_raw.drop("src_subject_id", axis=1)

In [5]:
#read form abcd_y_lt.csv
abcd_y_lt = pd.read_csv("G:\ABCD\\data\\abcd_y_lt.csv")
twins = abcd_y_lt[["src_subject_id", "rel_family_id"]]
twins_unique = twins.drop_duplicates(subset='src_subject_id', keep='first')
# 假设 data 是另一个包含 src_subject_id 和其他列的 DataFrame
merged_data = pd.merge(twins_unique, data_raw, on="src_subject_id", how="inner")
# 假设你的 DataFrame 叫做 df，且 rel_family_id 是你要分析的列
value_counts = merged_data['rel_family_id'].value_counts()

# 找出所有不同的重复次数
unique_counts = value_counts.value_counts().index.sort_values()

# 循环输出每个重复次数的数量
for count in unique_counts:
    count_of_duplicates = (value_counts == count).sum()
    print(f"{count}个相同值的数量: {count_of_duplicates}")

data = merged_data.groupby('rel_family_id').apply(lambda x: x.sample(1)).reset_index(drop=True)

C:\Users\Makka Papa\AppData\Local\Temp\ipykernel_7468\2767713118.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  abcd_y_lt = pd.read_csv("G:\ABCD\\data\\abcd_y_lt.csv")


1个相同值的数量: 7590
2个相同值的数量: 1701
3个相同值的数量: 54
4个相同值的数量: 2


C:\Users\Makka Papa\AppData\Local\Temp\ipykernel_7468\2767713118.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data = merged_data.groupby('rel_family_id').apply(lambda x: x.sample(1)).reset_index(drop=True)


In [9]:
data = data.drop(columns=["src_subject_id", "rel_family_id"])

# autoencoder

In [87]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

# 1. Data Normalization
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(data)

# 2. Define a Simplified Autoencoder Structure
input_dim = data.shape[1]
encoding_dim = 5  # Capturing five main features

# Input Layer
input_layer = Input(shape=(input_dim,))

# Encoding Layer
encoded = Dense(encoding_dim, activation='relu')(input_layer)

# Decoding Layer
decoded = Dense(input_dim, activation='linear')(encoded)

# Construct Autoencoder Model
autoencoder = Model(inputs=input_layer, outputs=decoded)

# Encoder Model (to extract features)
encoder = Model(inputs=input_layer, outputs=encoded)

# 3. Compile and Train the Model
autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

# Use EarlyStopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)

# Train the model
autoencoder.fit(data_scaled, data_scaled, epochs=100, batch_size=32, shuffle=True, validation_split=0.2, callbacks=[early_stopping])

# 4. Extract Main Components
encoded_data = encoder.predict(data_scaled)

# Convert the extracted components to a DataFrame
encoded_df = pd.DataFrame(encoded_data, columns=[f'PC{i+1}' for i in range(encoding_dim)])

# View the extracted components
print(encoded_df.head())


Epoch 1/100
280/280 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - loss: 0.0409 - mae: 0.1069 - val_loss: 0.0285 - val_mae: 0.0976
Epoch 2/100
280/280 ━━━━━━━━━━━━━━━━━━━━ 0s 822us/step - loss: 0.0293 - mae: 0.0998 - val_loss: 0.0272 - val_mae: 0.0928
Epoch 3/100
280/280 ━━━━━━━━━━━━━━━━━━━━ 0s 800us/step - loss: 0.0271 - mae: 0.0934 - val_loss: 0.0255 - val_mae: 0.0900
Epoch 4/100
280/280 ━━━━━━━━━━━━━━━━━━━━ 0s 822us/step - loss: 0.0254 - mae: 0.0908 - val_loss: 0.0246 - val_mae: 0.0885
Epoch 5/100
280/280 ━━━━━━━━━━━━━━━━━━━━ 0s 829us/step - loss: 0.0251 - mae: 0.0904 - val_loss: 0.0242 - val_mae: 0.0881
Epoch 6/100
280/280 ━━━━━━━━━━━━━━━━━━━━ 0s 786us/step - loss: 0.0248 - mae: 0.0896 - val_loss: 0.0241 - val_mae: 0.0876
Epoch 7/100
280/280 ━━━━━━━━━━━━━━━━━━━━ 0s 834us/step - loss: 0.0245 - mae: 0.0889 - val_loss: 0.0240 - val_mae: 0.0871
Epoch 8/100
280/280 ━━━━━━━━━━━━━━━━━━━━ 0s 852us/step - loss: 0.0248 - mae: 0.0897 - val_loss: 0.0239 - val_mae: 0.0868
Epoch 9/100
280/280 ━━━━━━━━━━━━━━

In [104]:
import numpy as np
from sklearn.metrics import mean_squared_error

# 用训练好的Autoencoder对原始数据进行重构
reconstructed_data = autoencoder.predict(data_scaled)

# 计算每个特征的总方差
total_variance = np.var(data_scaled, axis=0)

# 计算重构误差 (MSE)
reconstruction_error = mean_squared_error(data_scaled, reconstructed_data)

# 计算解释的方差比例
variance_explained_total = 1 - (reconstruction_error / np.mean(total_variance))

print(f'Variance Explained: {variance_explained_total:.4f}')


349/349 ━━━━━━━━━━━━━━━━━━━━ 0s 487us/step
Variance Explained: 0.3521


In [106]:
import numpy as np

# 假设你的负荷矩阵是 loading_matrix，形状为 (n_features, n_factors)
loading_matrix = np.array(encoded_df)  # 替换为你的实际负荷矩阵

# 计算每个因子的解释方差
variance_explained = np.sum(loading_matrix ** 2, axis=0)

# 将解释方差转换为百分比
total_variance = np.sum(variance_explained)
variance_explained_ratio = (variance_explained / total_variance) * 100

# 输出每个因子的解释方差百分比
for i, ratio in enumerate(variance_explained_ratio):
    # print(f'Factor {i+1} Variance Explained: {ratio:.2f}%')
    variance_explained_each = ratio*variance_explained_total
    print(f'Factor {i+1} Variance Explained: {variance_explained_each:.2f}%')


Factor 1 Variance Explained: 8.85%
Factor 2 Variance Explained: 4.27%
Factor 3 Variance Explained: 5.52%
Factor 4 Variance Explained: 6.78%
Factor 5 Variance Explained: 9.80%


In [76]:
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from math import sqrt

# Load data from dataframe
d = data.shape[1]  # Input dimension based on dataframe
X = data.values  # Convert dataframe to numpy array

# Define a function to build a deep autoencoder model
def build_autoencoder(input_dim, J):
    # Input layer
    input_layer = Input(shape=(input_dim,))
    
    # Encoder layers
    encoded = Dense(8 * J, activation='relu')(input_layer)
    bottleneck = Dense(J, activation='relu')(encoded)
    
    # Decoder layers
    decoded = Dense(8 * J, activation='relu')(bottleneck)
    output_layer = Dense(input_dim, activation='linear')(decoded)
    
    # Define the autoencoder model
    autoencoder = Model(inputs=input_layer, outputs=output_layer)
    return autoencoder

# Grid search for optimal J (1 <= J <= 10)
min_rmse = float('inf')
optimal_J = None
kf = KFold(n_splits=10, shuffle=True, random_state=42)

for J in range(1, 11):
    rmses = []
    for train_index, val_index in kf.split(X):
        X_train, X_val = X[train_index], X[val_index]
        
        # Build and compile the model
        autoencoder = build_autoencoder(input_dim=d, J=J)
        autoencoder.compile(optimizer=Adam(), loss='mean_squared_error')
        
        # Train the model
        autoencoder.fit(X_train, X_train, epochs=100, batch_size=64, verbose=0, validation_data=(X_val, X_val))
        
        # Predict on validation set
        X_val_pred = autoencoder.predict(X_val)
        
        # Calculate RMSE
        rmse = sqrt(mean_squared_error(X_val, X_val_pred))
        rmses.append(rmse)
    
    # Average RMSE for current J
    avg_rmse = np.mean(rmses)
    print(f'J = {J}, Average RMSE: {avg_rmse}')
    
    # Update optimal J
    if avg_rmse < min_rmse:
        min_rmse = avg_rmse
        optimal_J = J

print(f'Optimal J: {optimal_J}, Minimum RMSE: {min_rmse}')

# Build the final model with the optimal J
autoencoder = build_autoencoder(input_dim=d, J=optimal_J)
autoencoder.compile(optimizer=Adam(), loss='mean_squared_error')

# Train the final model on the entire dataset
autoencoder.fit(X, X, epochs=100, batch_size=64, verbose=1)


35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
J = 1, Average RMSE: 0.33886045973714163
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
J = 2, Average RMSE: 0.3257940067190376
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
35/35 ━

In [77]:
# 使用训练好的模型进行预测
X_reconstructed = autoencoder.predict(X)

# 使用 explained_variance_score 计算解释方差
from sklearn.metrics import explained_variance_score

explained_variance = explained_variance_score(X, X_reconstructed)
print(f'Explained Variance Score: {explained_variance:.4f}')


349/349 ━━━━━━━━━━━━━━━━━━━━ 0s 690us/step
Explained Variance Score: 0.3687


# EFA from previous research

In [9]:
# delete columns with low frequency (more than 99.5% of the values are 0)
low_frequency_columns = data.columns[data.apply(lambda col: (col == 0).mean() > 0.995)]
data_cleaned = data.drop(columns=low_frequency_columns)
print(f"Removed columns with low frequency: {low_frequency_columns.tolist()}")

#load corerlation matrix from polychoric correlation matrix

correlation_matrix = pd.read_csv(data_path + "/factor analysis/output/polychoric_correlation_matrix_twins.csv", index_col=0)

# mark highly correlated pairs (r > 0.75)
high_corr_pairs = (correlation_matrix.abs() > 0.75).where(lambda x: np.triu(x, 1)).stack().index.tolist()
print(f"Highly correlated pairs (r > 0.75): {high_corr_pairs}")

Removed columns with low frequency: ['cbcl_q02_p', 'cbcl_q73_p', 'cbcl_q99_p', 'cbcl_q101_p', 'cbcl_q105_p']
Highly correlated pairs (r > 0.75): [('cbcl_q08_p', 'cbcl_q10_p'), ('cbcl_q08_p', 'cbcl_q78_p'), ('cbcl_q16_p', 'cbcl_q97_p'), ('cbcl_q20_p', 'cbcl_q21_p'), ('cbcl_q22_p', 'cbcl_q28_p'), ('cbcl_q23_p', 'cbcl_q28_p'), ('cbcl_q25_p', 'cbcl_q48_p'), ('cbcl_q40_p', 'cbcl_q70_p'), ('cbcl_q53_p', 'cbcl_q55_p'), ('cbcl_q56c_p', 'cbcl_q56f_p'), ('cbcl_q57_p', 'cbcl_q97_p'), ('cbcl_q81_p', 'cbcl_q82_p')]


In [10]:
from collections import defaultdict

def find_connected_groups(pairs):
    # 建立图结构
    graph = defaultdict(set)
    for col1, col2 in pairs:
        graph[col1].add(col2)
        graph[col2].add(col1)
    
    # 深度优先搜索（DFS）找到所有连通分量
    visited = set()
    connected_groups = []

    def dfs(node, group):
        visited.add(node)
        group.add(node)
        for neighbor in graph[node]:
            if neighbor not in visited:
                dfs(neighbor, group)

    # 遍历所有节点，找到每个连通分量
    for node in graph:
        if node not in visited:
            group = set()
            dfs(node, group)
            connected_groups.append(tuple(sorted(group)))

    return connected_groups

# 使用函数
result = find_connected_groups(high_corr_pairs)
print("number of connected groups:", len(result))
print("columns for each connected group:", result)

number of connected groups: 9
columns for each connected group: [('cbcl_q08_p', 'cbcl_q10_p', 'cbcl_q78_p'), ('cbcl_q16_p', 'cbcl_q57_p', 'cbcl_q97_p'), ('cbcl_q20_p', 'cbcl_q21_p'), ('cbcl_q22_p', 'cbcl_q23_p', 'cbcl_q28_p'), ('cbcl_q25_p', 'cbcl_q48_p'), ('cbcl_q40_p', 'cbcl_q70_p'), ('cbcl_q53_p', 'cbcl_q55_p'), ('cbcl_q56c_p', 'cbcl_q56f_p'), ('cbcl_q81_p', 'cbcl_q82_p')]


In [13]:
#create dataframe to store the final data
data_final = data_cleaned.copy()
for group in result:
    # calculate the average of the columns in the group
    data_final[f"avg_{'_'.join(group)}"] = data_cleaned[list(group)].mean(axis=1).round().astype(int)
    # delete the original columns
    data_final.drop(columns=list(group), inplace=True)

In [14]:
# #scale data_final
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# data_scaled = scaler.fit_transform(data_final)

# #make data_scaled to dataframe
# data_scaled = pd.DataFrame(data_scaled, columns=data_final.columns)
data_scaled = data_final

In [17]:
data_scaled.drop(columns=["src_subject_id", "rel_family_id"], inplace=True)

In [18]:
import pandas as pd
from factor_analyzer import FactorAnalyzer
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity, calculate_kmo
import matplotlib.pyplot as plt
import seaborn as sns


# Bartlett 和 KMO 测试
chi_square_value, p_value = calculate_bartlett_sphericity(data_scaled)
print(f"Bartlett's Test Chi-square: {chi_square_value}, p-value: {p_value}")
kmo_all, kmo_model = calculate_kmo(data_scaled)
print(f"KMO Test Score: {kmo_model}")

# factor analysis
fa = FactorAnalyzer(n_factors=16,  rotation= rotation, method = 'principal')
# fa = FactorAnalyzer(n_factors=16, rotation="equamax", method='principal')
# fa.fit(data_cleaned)
fa.fit(data_scaled)

# factor loadings
factor_loadings = fa.loadings_
# factor_loadings_df = pd.DataFrame(factor_loadings, columns=["Factor 1", "Factor 2", "Factor 3", "Factor 4", "Factor 5", "Factor 6"])
factor_loadings_df = pd.DataFrame(factor_loadings, columns=[f"Factor {i}" for i in range(1, 17)])
# factor_loadings_df = pd.DataFrame(factor_loadings, columns=["Factor 1", "Factor 2", "Factor 3", "Factor 4", "Factor 5"])
# factor_loadings_df.index = data_cleaned.columns
factor_loadings_df.index = data_final.columns
print("Factor Loadings DataFrame:\n", factor_loadings_df)

# variance explained
variance_explained = fa.get_factor_variance()
print("Variance Explained:\n", variance_explained)



Bartlett's Test Chi-square: 238146.69520761166, p-value: 0.0
KMO Test Score: 0.9624037123398348


c:\Anaconda\envs\PhD\Lib\site-packages\factor_analyzer\utils.py:244: UserWarning: The inverse of the variance-covariance matrix was calculated using the Moore-Penrose generalized matrix inversion, due to its determinant being at or very close to zero.
  warnings.warn(


Factor Loadings DataFrame:
                              Factor 1  Factor 2  Factor 3  Factor 4  Factor 5  \
cbcl_q01_p                   0.158746  0.054357 -0.137513  0.317554 -0.009661   
cbcl_q03_p                   0.583481 -0.001042  0.031194  0.142819  0.032724   
cbcl_q04_p                   0.147039 -0.034074 -0.041008  0.566630  0.037088   
cbcl_q05_p                   0.196349 -0.046049 -0.013099  0.202941  0.009528   
cbcl_q06_p                  -0.054184  0.004284 -0.008707 -0.017063  0.009467   
...                               ...       ...       ...       ...       ...   
avg_cbcl_q25_p_cbcl_q48_p    0.006006  0.040872  0.114927  0.026406  0.012949   
avg_cbcl_q40_p_cbcl_q70_p   -0.058122  0.004397  0.236370 -0.087217  0.070249   
avg_cbcl_q53_p_cbcl_q55_p   -0.021818  0.035933  0.024859 -0.082826 -0.033896   
avg_cbcl_q56c_p_cbcl_q56f_p  0.005402  0.015606 -0.031792 -0.036812  0.678849   
avg_cbcl_q81_p_cbcl_q82_p   -0.052846  0.014157  0.557623  0.051722 -0.048664   


In [13]:
variance_explained[1]

array([0.03245182, 0.01720842, 0.02686883, 0.01793263, 0.02223959,
       0.02282064, 0.01686017, 0.0254489 , 0.01819661, 0.01726673,
       0.02757612, 0.01682084, 0.02125505, 0.01760954, 0.01823394,
       0.01747789])

In [14]:
variance_explained[2][-1]

0.33626773081510103

In [15]:
for i in range(1, 17):
    # all loading values of the factor greater than 0.1
    factor_values = factor_loadings_df[f"Factor {i}"][factor_loadings_df[f"Factor {i}"] > 0.1]

    # print(f"Factor {i}:\n", factor_loadings_df[f"Factor {i}"].sort_values(ascending=False).head(20))

    # descending order
    print(f"Factor {i}:\n", factor_values.sort_values(ascending=False))

Factor 1:
 cbcl_q95_p                               0.576580
cbcl_q86_p                               0.562006
cbcl_q03_p                               0.533706
cbcl_q109_p                              0.519226
cbcl_q87_p                               0.509411
cbcl_q68_p                               0.483531
cbcl_q88_p                               0.474791
cbcl_q27_p                               0.386796
cbcl_q14_p                               0.377646
cbcl_q19_p                               0.361401
cbcl_q33_p                               0.354315
cbcl_q103_p                              0.268592
avg_cbcl_q22_p_cbcl_q23_p_cbcl_q28_p     0.265540
cbcl_q26_p                               0.244424
cbcl_q104_p                              0.235022
cbcl_q05_p                               0.181576
cbcl_q43_p                               0.178289
cbcl_q41_p                               0.166262
cbcl_q89_p                               0.155802
cbcl_q16_p                             

In [16]:
# filtered_columns = {}
result_df = pd.DataFrame()

for i in range(1, 17):
    # 筛选出符合条件的列名
    factor_values = factor_loadings_df[f"Factor {i}"][factor_loadings_df[f"Factor {i}"] > 0.1]
    # filtered_columns[f"Factor {i}"] = factor_values.index.tolist()  # 保存列名（或索引）
        # 创建一个临时数据框保存因子名、列名和加载值
    temp_df = pd.DataFrame({
        f"Factor {i} Variable": factor_values.index,     # 存储列名
        f"Factor {i} Loading": factor_values.values      # 存储加载值
    })
    
    # 将临时数据框合并到结果数据框
    result_df = pd.concat([result_df, temp_df], axis=1)


# # 保存为CSV文件
# filtered_columns_df.to_csv("filtered_factor_columns.csv", index=False)


# Edit and translate

In [30]:
import pandas as pd
from bs4 import BeautifulSoup
from googletrans import Translator
import re
import time

# 指定源语言和目标语言
translator = Translator()

# 解析 element.html 文件以获取列名和详细信息
with open("data/element.html", "r", encoding="utf-8") as file:
    soup = BeautifulSoup(file, "html.parser")

# 创建一个字典来存储列名和对应的详细信息
column_details = {}
result_df = pd.DataFrame()

# 提取 cbcl_q 列名的正则表达式
cbcl_pattern = re.compile(r"(cbcl_q\d+[a-z]*_p)")

for i in range(1, 17):
    # 筛选出符合条件的加载值
    factor_values = factor_loadings_df[f"Factor {i}"][factor_loadings_df[f"Factor {i}"] > 0.1]
    
    original_text = []
    translated_text = []
    for column_name in factor_values.index:
        # 查找 column_name 中的所有 cbcl_q 字段
        cbcl_items = cbcl_pattern.findall(column_name)  # 提取所有符合 cbcl_qXX_p 或 cbcl_qXXh_p 格式的子串

        # 初始化存储每个 cbcl 字段详细信息的列表
        original = []
        details = []
        for cbcl_item in cbcl_items:
            # 获取每个 cbcl 字段的详细信息
            target = soup.find(lambda tag: tag.name == "td" and cbcl_item in tag.get_text(strip=True))
            if target:
                detail_info = target.find_next("td").get_text(strip=True)
                # 保存原始详细信息
                original.append(detail_info)
                
                # 翻译详细信息并添加到结果
                try:
                    translated_detail = translator.translate(detail_info, src='es', dest='en').text
                except AttributeError as e:
                    print(f"An error occurred: {e}")
                    translated_detail = detail_info
                details.append(translated_detail)
                time.sleep(0.25)

        # 将所有细节合并为单个字符串，并添加到列表中
        original_text.append("; ".join(original) if original else "N/A")
        translated_text.append("; ".join(details) if details else "N/A")

    # 创建一个临时数据框保存因子名、列名、加载值和详细信息
    temp_df = pd.DataFrame({
        f"Factor {i} Variable": factor_values.index,  # 存储列名
        f"Factor {i} Loading": factor_values.values,  # 存储加载值
        f"Factor {i} Detail": original_text,  # 映射详细信息
        f"Factor {i} Translated_Detail": translated_text  # 映射翻译后详细信息
    })

    # 按加载值降序排序
    sorted_df = temp_df.sort_values(by=f"Factor {i} Loading", ascending=False).reset_index(drop=True)
    # 将临时数据框合并到结果数据框
    result_df = pd.concat([result_df.reset_index(drop=True), sorted_df.reset_index(drop=True)], axis=1)

# 保存结果为CSV文件
result_df.to_csv(output_dir + "/interpretable_information_EN.csv", index=False)


In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
from googletrans import Translator
import re
import time

# 指定源语言和目标语言
translator = Translator()

# 解析 element.html 文件以获取列名和详细信息
with open("data/element.html", "r", encoding="utf-8") as file:
    soup = BeautifulSoup(file, "html.parser")

# 创建一个字典来存储列名和对应的详细信息
column_details = {}
result_df = pd.DataFrame()

# 提取 cbcl_q 列名的正则表达式
cbcl_pattern = re.compile(r"(cbcl_q\d+[a-z]*_p)")

for i in range(1, 17):
    # 筛选出符合条件的加载值
    factor_values = factor_loadings_df[f"Factor {i}"][factor_loadings_df[f"Factor {i}"] > 0.1]
    
    original_text = []
    translated_text = []
    for column_name in factor_values.index:
        # 查找 column_name 中的所有 cbcl_q 字段
        cbcl_items = cbcl_pattern.findall(column_name)  # 提取所有符合 cbcl_qXX_p 或 cbcl_qXXh_p 格式的子串

        # 初始化存储每个 cbcl 字段详细信息的列表
        original = []
        details = []
        for cbcl_item in cbcl_items:
            # 获取每个 cbcl 字段的详细信息
            target = soup.find(lambda tag: tag.name == "td" and cbcl_item in tag.get_text(strip=True))
            if target:
                detail_info = target.find_next("td").get_text(strip=True)
                # 保存原始详细信息
                original.append(detail_info)
                
                # 翻译详细信息并添加到结果
                try:
                    translated_detail = translator.translate(detail_info, src='es', dest='zh-cn').text
                except AttributeError as e:
                    print(f"An error occurred: {e}")
                    translated_detail = detail_info
                details.append(translated_detail)
                time.sleep(0.25)

        # 将所有细节合并为单个字符串，并添加到列表中
        original_text.append("; ".join(original) if original else "N/A")
        translated_text.append("; ".join(details) if details else "N/A")

    # 创建一个临时数据框保存因子名、列名、加载值和详细信息
    temp_df = pd.DataFrame({
        f"Factor {i} Variable": factor_values.index,  # 存储列名
        f"Factor {i} Loading": factor_values.values,  # 存储加载值
        f"Factor {i} Detail": original_text,  # 映射详细信息
        f"Factor {i} Translated_Detail": translated_text  # 映射翻译后详细信息
    })

    # 按加载值降序排序
    sorted_df = temp_df.sort_values(by=f"Factor {i} Loading", ascending=False).reset_index(drop=True)
    # 将临时数据框合并到结果数据框
    result_df_CN = pd.concat([result_df.reset_index(drop=True), sorted_df.reset_index(drop=True)], axis=1)

# 保存结果为CSV文件
result_df_CN.to_csv(output_dir + "/interpretable_information_CN.csv", index=False)


In [32]:
EN_pattern = re.compile(r"Factor \d+ Translated_Detail")
selected_columns = [col for col in result_df.columns if EN_pattern.match(col)]

result_df[selected_columns].to_csv(output_dir + "/details.csv", index=False)

# Generate labels with the factors

In [19]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=3, random_state=42)
combined_label_cluster = kmeans.fit_predict(factor_scores_df[['Factor1', 'Factor2', 'Factor3']])


NameError: name 'factor_scores_df' is not defined